In [ ]:
!pip install scipy qutip sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 58.0 MB/s eta 0:00:00


# Code

## Hamiltonian Generation

In [ ]:

import numpy as np
import re
import sympy
from sympy.parsing.latex import parse_latex
import sympy as sp

class BKP_Hamiltonian:
    def __init__(self, values, weights, max_weight, penalty=True):

        self.weights     = weights
        self.values      = values
        self.max_weight  = max_weight
        self.do_penalty  = penalty

        self.Hamiltonian = None
        self.Hc          = None
        self.Hp          = None
        self.n_qubits    = None
        self.symbols     = None
        self.penalty     = None
        self._get_BKP_Hamiltonian(penalty=penalty)
        self._binary_to_pauli_list()

    def _prepare_slack_variables(self):
        max_weight = self.max_weight
        n_slack_vars = int(np.floor(np.log2(max_weight))) + 1
        slack_vars = sympy.symbols(r's_{{:{}}}'.format(n_slack_vars))

        slack_expr = 0.

        coeffs = []
        vars = []

        for ii in range(len(slack_vars)-1):
            coeffs.append(2**ii)
            vars.append(slack_vars[ii])

        # Now we must get the prefactor for the final slack variable:
        # This comes from the 10 - Upper_Bound(BinaryExpansion(n-1 slack variables))
        final_coeff = max_weight - sum(coeffs)
        coeffs.append(final_coeff)
        vars.append(slack_vars[-1])
        # Return data as dictionary with coeffs, variables
        output_dict = {}
        for ii in range(len(coeffs)):
            output_dict[ii] = {'Coefficient': coeffs[ii], 'Variable': vars[ii]}
        return(output_dict)


    def _get_BKP_Hamiltonian(self, penalty=False):

        values = self.values
        weights = self.weights
        max_weight = self.max_weight

        # Get the number of items
        n_items = len(values)
        # Number of slack variables
        n_slack = int(np.floor(np.log2(max_weight))) + 1
        # Number of binary variables/qubits:
        n_qubits = n_items + n_slack
        self.n_qubits = n_qubits
        # Prepare some symbols:
        bin_vars = sympy.symbols(r'x_{{:{}}}'.format(n_qubits))
        self.symbols = list(bin_vars)
        l_val = sympy.symbols(r'\lambda')

        # Create a symbol dictionary to ensure we are accessing the correct symbol
        keys = [ii for ii in range(n_qubits)]
        symbol_dict = dict(zip(keys, bin_vars))

        # Make sure we have values and weights as lists
        if type(values) != type([1]):
            values = values.tolist()
        if type(weights) != type([1]):
            weights = weights.tolist()

        # Calculate the slack variable terms
        slack_vars = self._prepare_slack_variables()
        # Get slack variable weights from the slack_vars dictionary
        slack_var_weights = list(map(lambda key: slack_vars[key]['Coefficient'],
                                    slack_vars.keys()))
        all_values = values + [0] * n_slack
        all_weights = weights + slack_var_weights

        # Calculate the Problem Hamiltonian:
        Hp = 0.
        for ii in range(n_qubits):
            Hp = Hp + all_values[ii] * symbol_dict[keys[ii]]

        # Construct the Cost Hamiltonian:
        Hc = 0.
        for ii in range(n_qubits):
            Hc = Hc + all_weights[ii] * symbol_dict[keys[ii]]
        Hc = Hc - max_weight
        self.Hp = Hp
        self.Hc = Hc
        # Construct the Full Hamiltonian:
        Hfull = - Hp + l_val * (Hc**2).expand()
        # Now, we can make a suggestion for the value of lambda (our penalty factor):
        # Linear Bounds:
        lin_upper_bound = sum(all_values)
        lin_lower_bound = 0. # sum of weighted binary vars means everything will be 0 if all x_i = 0
        suggested_penalty = 1 + (lin_upper_bound)
        print("Suggested Penalty factor is: {}\n".format(suggested_penalty))
        self.penalty = suggested_penalty
        if type(penalty) != bool:
            print('Using a different penalty than suggested!')
            self.Hamiltonian = Hfull.subs({l_val : penalty})
        elif penalty == True:
            self.Hamiltonian = Hfull.subs({l_val : suggested_penalty})
            return
        else:
            self.Hamiltonian = Hfull
            return


    def _binary_to_qubit_ham(self, include_id=False):
        """
        Map a symbolic binary Hamiltonian to a spin Hamiltonian.

        Arguments:
        H_bin -- The binary Hamiltonian as a SymPy object
        symbol_list -- SymPy symbols defining the Hamiltonian
        include_id -- identity as symbol (True) or value 1 (False)
        """

        H_bin = self.Hamiltonian
        symbol_list = self.symbols
        # Initialize spin variables (Z0, Z1, ..., Zn)
        z_symbols = sp.symbols('z:{}'.format(len(symbol_list)))
        self.spin_symbols = z_symbols
        # Define the identity operator (I_j)
        Ident = sp.symbols(r'\mathbb{I}') if include_id else 1.0

        # Create a mapping dictionary from binary symbols to spin expressions
        bin2spin_dict = {
            symbol: (1/2)*(Ident - z) for symbol, z in zip(symbol_list, z_symbols)
        }

        # Convert the binary Hamiltonian to a spin Hamiltonian
        spin_ham = H_bin.subs(bin2spin_dict).expand()

        # Z^2 = I
        sq_z = [z**2 for z in z_symbols]
        sq_values = [Ident] * len(z_symbols)  # All squared terms map to the identity
        spin_squared_map = dict(zip(sq_z, sq_values))

        # Substitute squared terms
        red_spin_ham = spin_ham.subs(spin_squared_map)
        self.Spin_Hamiltonian = red_spin_ham
        return


    def _check_spinz(self, input_list, spinz):
        out_val = ['I']*len(spinz)
        for ll in range(len(input_list)):
            out_val[int(input_list[ll].strip('z'))] = 'Z'
        return out_val


    def _sympy_to_pauli_dict(self):
        """
        Convert a sympy spin Hamiltonian expression to a dictionary with
        Pauli words as keys and string coefficients as values.
        """
        smpy_exp = self.Spin_Hamiltonian
        # Determine the number of qubits
        spinz = smpy_exp.free_symbols

        # Split at spaces so we have the individual terms/coefficients
        split_expr = str(smpy_exp).split()

        # Firs iteration
        matrix_dict = {}
        split_term = split_expr[0].split('*')
        tmp_coeff = split_term[0]
        tmp_paulis = split_term[1:]
        pauli_word = ''.join(self._check_spinz(tmp_paulis, spinz))
        matrix_dict[pauli_word] = tmp_coeff

        # Iterate through the remaining terms
        for ii in range(1, len(split_expr), 2):
            tmp_sign = split_expr[ii]
            split_term = split_expr[ii+1].split('*')
            tmp_coeff  = split_term[0]
            tmp_paulis = split_term[1:]
            pauli_word = ''.join(self._check_spinz(tmp_paulis, spinz))
            matrix_dict[pauli_word] = tmp_sign+tmp_coeff
        self.pauli_dict = matrix_dict
        return matrix_dict


    def _binary_to_pauli_list(self):
        """
        Maps a binary Hamiltonian to Pauli terms and coefficients.

        Arguments:
        H_total -- The binary Hamiltonian
        symbol_list -- symbols defining the Hamiltonian
        """
        spin_ham = self._binary_to_qubit_ham()
        op_dict = self._sympy_to_pauli_dict()
        pauli_list = [[key, float(value)] for key, value in op_dict.items()]
        self.Pauli_List = pauli_list
        return pauli_list


def make_pauli_dict(pauli_list):
    pauli_words = []
    pauli_coeff = []
    for ii in range(len(pauli_list)):
        pauli_words.append(pauli_list[ii][0])
        pauli_coeff.append(pauli_list[ii][1])
    return(dict(zip(pauli_words, pauli_coeff)))


def vec_query(arr, my_dict):
    '''
    This function vectorizes dictionary querying.
    It allows us to query `my_dict` with a np.array `arr` of keys.
    This avoids a loop through the list of keys.
    '''
    return np.vectorize(my_dict.__getitem__, otypes=[tuple])(arr)


def build_hamiltonian(pauli_dict):

    X = qt.Qobj([[0,1],[1,0]])
    Y = qt.Qobj([[0,complex(0,-1)],[complex(0,1),0]])
    Z = qt.Qobj([[1,0],[0,-1]])
    I = qt.qeye(dimensions=2)
    # Define a dictionary with the four Pauli matrices:
    pms = {'I': I,'X': X,'Y': Y,'Z': Z}

    Ham_out = 0.0

    pauli_strings = list(pauli_dict.keys())

    for tmp_pauli in pauli_strings:
        coeff = pauli_dict[tmp_pauli]
        pauli_word = vec_query(list(tmp_pauli), pms)
        word_matrix = qt.tensor(pauli_word)
        Ham_out += coeff * word_matrix
    return(Ham_out)



def get_mixer(Nq):
    sx = qt.sigmax()
    sy = qt.sigmay()
    II = qt.qeye(2)
    Hm = 0.0
    for ii in range(Nq):
        Hm += tensor([sx if jj == ii else II for jj in range(Nq)])
    return(Hm)


# Plotting/Data Functions
import numpy as np
import pandas as pd
import os
import qutip as qt
from qutip import tensor, basis, sigmax, sigmaz, qeye, expect
from scipy.optimize import minimize
from tqdm.auto import trange


# Plot Formatting:

#----------- Plot Formatting ---------#
import matplotlib_inline.backend_inline
# from IPython.display import Image, HTML
import matplotlib.pyplot as plt
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 100
%matplotlib inline

## Optimization Code

## Calculation

In [ ]:

values = [2, 5, 7, 3]
weights = [2.5, 3, 4, 3.5]
max_weight = 7

my_ham = BKP_Hamiltonian(values, weights, max_weight, penalty=2)

ham_dict = make_pauli_dict(my_ham.Pauli_List)

Suggested Penalty factor is: 18

Using a different penalty than suggested!


In [ ]:
# Define QAOA circuit for given angles
def qaoa_circuit(gamma, beta, Hc, Hm):
    """Applies the QAOA circuit with p layers using parameters gamma and beta."""
    state = initial_state
    for k in range(pl):
        # Apply problem unitary e^{-i * gamma * Hc}
        state = (-1j * abs(gamma[k]) * Hc).expm() * state
        # Apply mixing unitary e^{-i * beta * Hm}
        state = (-1j * abs(beta[k]) * Hm).expm() * state
    return state


# Objective function to minimize
def objective(params, Hc, Hm):
    """Objective function to minimize: expectation value of Hc."""
    plen = int(len(params) / 2)
    gamma = params[:plen]
    beta = params[plen:]
    final_state = qaoa_circuit(gamma, beta, Hc, Hm)
    return expect(Hc, final_state)  # Expectation value of Hc

In [ ]:
n_iterations = 50
# Define Parameters
Nq = len(list(ham_dict.keys())[0]) # Number of qubits (or spins)
Hm = get_mixer(Nq)

# Define Pauli matrices and identity
sx = sigmax()
sz = sigmaz()
II = qeye(2)

Hc = build_hamiltonian(ham_dict)
energies, wfns = Hc.eigenstates()
offset = abs(energies[0])
Hc += offset*qt.tensor([II]*Nq)
outdir = "BFGS/{}"
bfgs_results_layers = {}
layers_tests = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]

for nl in layers_tests:
    results_iterations = {}
    pl = nl # Number of QAOA layers
    print('Running for {} Layers: '.format(nl))
    Hm = get_mixer(Nq)
    new_savedir = outdir.format('{}-Layers/'.format(pl))
    os.makedirs(new_savedir, exist_ok=True)
    per_layer_energies = []
    for run in trange(n_iterations):

        # Initial state: uniform superposition |+> for all qubits
        initial_state = tensor([basis(2, 0) + basis(2, 1) for _ in range(Nq)]).unit()
        state = initial_state
        # Initial guess for parameters gamma and beta
        initial_params = np.random.rand(2 * pl) * np.pi
        result = minimize(objective, initial_params, args=(Hc, Hm), method="BFGS")

        optimal_params = result.x
        optimal_energy = result.fun
        optimal_state = qaoa_circuit(optimal_params[:pl], optimal_params[pl:], Hc, Hm)
        gs_overlap = abs(optimal_state.overlap(wfns[0]))
        # Output results
        print("Run Number {}".format(run))
        print("Optimal Parameters (gamma, beta): {}".format(optimal_params))
        print("Optimal Energy (Ground State): {}".format(optimal_energy))
        print("Approximation Ratio: {}\n".format(gs_overlap))
        results_iterations[run] = {'Optimal State': optimal_state,
                                   'Energy': optimal_energy,
                                   'Approximation Ratio': gs_overlap}

        save_name = new_savedir + "QAOA-Knapsack-FullPenalty-Run{}.npz".format(run)
        per_layer_energies.append(optimal_energy)
        np.savez_compressed(save_name, optimal_state=optimal_state,
                            optimal_energy=optimal_energy, overlap=gs_overlap,
                            n_iter=result.nit, n_fev=result.nfev, optimal_angles=result.x)
    bfgs_results_layers[nl] = results_iterations


Running for 20 Layers: 


  0%|          | 0/50 [00:00<?, ?it/s]

Run Number 0
Optimal Parameters (gamma, beta): [ 0.36624313  2.32894574  1.43193039  2.61400032  0.5909554   0.87317072
  3.01242409  1.34252931  1.26355183  2.93108012  2.75028526  1.53303423
  2.79000764  1.02532213 -0.05903015  2.70328958  1.63229503  2.95871499
  2.2171208   0.21026703  2.42066559  2.49688937  2.52342931  1.29422183
  1.6623497   2.87393879  2.30858923  0.41151669  2.54799883  0.7672491
  1.80374975  1.11574522  2.83973188  1.18974034  1.40947235  1.48901919
  0.65182336  2.49609302  0.49872788  2.68118963]
Optimal Energy (Ground State): 17.309380490876567
Approximation Ratio: 0.0811009962294534

Run Number 1
Optimal Parameters (gamma, beta): [1.35551751 0.79216774 1.62655227 1.27618223 1.08311064 1.84795699
 1.28017583 2.53478747 1.32756908 1.06379035 2.73812128 0.68432534
 1.13029945 2.23242216 2.39239282 2.88066901 2.29026067 0.42951777
 0.29893842 2.5379252  1.31673565 0.83192013 1.33253745 0.36605469
 0.73010765 1.89173411 0.89371129 2.60422176 2.19742453 0.47